In [13]:
!nvidia-smi

Tue May  6 20:11:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.230.02             Driver Version: 535.230.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GH200 120GB             On  | 00000009:01:00.0 Off |                    0 |
| N/A   28C    P0             138W / 900W |  37751MiB / 97871MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import json

import torch
import transformers
from environs import env
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from local_funcs import prompt_funcs
from yiutils.project_utils import find_project_root

proj_root = find_project_root("justfile")
data_dir = proj_root / "data"

print(transformers.__version__)
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

env.read_env(proj_root / ".env")
access_token = env("HUGGINGFACE_TOKEN")

path_to_mr_pubmed_data = (
    data_dir / "intermediate" / "mr-pubmed-data" / "mr-pubmed-data.json"
)
assert path_to_mr_pubmed_data.exists(), (
    f"Data file {path_to_mr_pubmed_data} does not exist."
)

with open(path_to_mr_pubmed_data, "r") as f:
    mr_pubmed_data = json.load(f)

article_data = mr_pubmed_data[0]

message_metadata = prompt_funcs.make_message_metadata(article_data["ab"])
message_results = prompt_funcs.make_message_results(article_data["ab"])


/home/b5a/ik18445.b5a/micromamba/envs/data-extraction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3
2.6.0
True
12.6


In [3]:
MODEL_ID = "meta-llama/Llama-3.2-11B-Vision-Instruct"

device = "auto"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map=device,
    token=access_token,
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.39it/s]


In [9]:
# messages = message_metadata
messages = message_results
input_ids = tokenizer.apply_chat_template(
    conversation=messages, add_generation_prompt=True, return_tensors="pt"
).to(model.device)
print(input_ids.shape)
input_ids

torch.Size([1, 825])


tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   2705,   3297,    220,   2366,     20,    271,   2675,    527,
            264,    828,  28568,   8647,    369,  60508,  13687,   2038,    505,
           3495,  16064,     13,   1472,   4320,   1855,   3488,    449,    264,
           3254,   4823,    925,     13, 128009, 128006,    882, 128007,   1432,
            394,   1115,    374,    459,   8278,    505,    264,  46211,  70664,
           4288,   2065,   4007,    627,    504,    330,   2149,  11353,  15652,
          12207,  25949,   8624,  23104,    323,    706,   1027,  10815,    311,
           5370,  19338,    304,  90380,   7978,     13,   4452,     11,  16195,
           8999,  19415,    278,  73279,   1701,  46211,  70664,   4288,   2065,
            320,  18953,      8,    311,  21635,  16558,  12912,    527,   7347,
             13,   1226,  20

In [10]:
terminators = [
    tokenizer.eos_token_id,
    # tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]
outputs = model.generate(
    input_ids,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.1,
    # top_p=0.95,
)
print(outputs.shape)
outputs

torch.Size([1, 1824])


tensor([[128000, 128006,   9125,  ...,    457,     92, 128009]],
       device='cuda:0')

In [11]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'system\n\nCutting Knowledge Date: December 2023\nToday Date: 06 May 2025\n\nYou are a data scientist responsible for extracting accurate information from research papers. You answer each question with a single JSON string.user\n\n\n                This is an abstract from a Mendelian randomization study.\n                    "Alcohol consumption significantly impacts disease burden and has been linked to various diseases in observational studies. However, comprehensive meta-analyses using Mendelian randomization (MR) to examine drinking patterns are limited. We aimed to evaluate the health risks of alcohol use by integrating findings from MR studies. A thorough search was conducted for MR studies focused on alcohol exposure. We utilized two sets of instrumental variables-alcohol consumption and problematic alcohol use-and summary statistics from the FinnGen consortium R9 release to perform de novo MR analyses. Our meta-analysis encompassed 64 published and 151 de novo MR analyses acro

In [12]:
response = outputs[0][input_ids.shape[-1] :]
result = tokenizer.decode(response, skip_special_tokens=True)
print(result)

{
    "results": [
        {
            "exposure": "Genetic predisposition to alcohol consumption",
            "outcome": "Parkinson's disease",
            "beta": null,
            "units": null,
            "odds ratio": null,
            "hazard ratio": null,
            "95% CI": null,
            "SE": null,
            "P-value": null,
            "direction": "decreases"
        },
        {
            "exposure": "Genetic predisposition to alcohol consumption",
            "outcome": "Prostate hyperplasia",
            "beta": null,
            "units": null,
            "odds ratio": null,
            "hazard ratio": null,
            "95% CI": null,
            "SE": null,
            "P-value": null,
            "direction": "decreases"
        },
        {
            "exposure": "Genetic predisposition to alcohol consumption",
            "outcome": "Rheumatoid arthritis",
            "beta": null,
            "units": null,
            "odds ratio": null,
           